In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

import torch

In [20]:
df = pd.read_csv("../input/xeno-canto-bird-recordings-extended-a-m/train_extended.csv")
df

,rating,playback_used,ebird_code,channels,date,duration,filename,species,title,secondary_labels,...,background,xc_id,url,country,author,primary_label,longitude,time,recordist,license
0,0.0,no,aldfly,2 (stereo),2019-06-11,49,XC554809.mp3,Alder Flycatcher,XC554809 Alder Flycatcher (Empidonax alnorum),[],...,NaN,554809,https://www.xeno-canto.org/554809,United States,Ron Overholtz,Empidonax alnorum_Alder Flycatcher,-149.9757,07:00,Ron Overholtz,Creative Commons Attribution-NonCommercial-Sha...
1,0.0,no,aldfly,2 (stereo),2020-04-14,21,XC552408.mp3,Alder Flycatcher,XC552408 Alder Flycatcher (Empidonax alnorum),[],...,NaN,552408,https://www.xeno-canto.org/552408,Colombia,Jerome Fischer,Empidonax alnorum_Alder Flycatcher,-74.6520,17:00,Jerome Fischer,Creative Commons Attribution-NonCommercial-Sha...
2,0.0,yes,aldfly,2 (stereo),2020-04-05,35,XC544552.mp3,Alder Flycatcher,XC544552 Alder Flycatcher (Empidonax alnorum),[],...,NaN,544552,https://www.xeno-canto.org/544552,Colombia,Jerome Fischer,Empidonax alnorum_Alder Flycatcher,-74.6520,07:00,Jerome Fischer,Creative Commons Attribution-NonCommercial-Sha...
3,0.0,yes,aldfly,2 (stereo),2020-04-05,12,XC544551.mp3,Alder Flycatcher,XC544551 Alder Flycatcher (Empidonax alnorum),[],...,NaN,544551,https://www.xeno-canto.org/544551,Colombia,Jerome Fischer,Empidonax alnorum_Alder Flycatcher,-74.6520,07:00,Jerome Fischer,Creative Commons Attribution-NonCommercial-Sha...
4,0.0,yes,aldfly,2 (stereo),2020-04-05,38,XC544550.mp3,Alder Flycatcher,XC544550 Alder Flycatcher (Empidonax alnorum),[],...,NaN,544550,https://www.xeno-canto.org/544550,Colombia,Jerome Fischer,Empidonax alnorum_Alder Flycatcher,-74.6520,07:00,Jerome Fischer,Creative Commons Attribution-NonCommercial-Sha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23779,4.5,no,yetvir,2 (stereo),2020-05-16,63,XC558915.mp3,Yellow-throated Vireo,XC558915 Yellow-throated Vireo (Vireo flavifrons),['Setophaga striata_Blackpoll Warbler'],...,Blackpoll Warbler (Setophaga striata),558915,https://www.xeno-canto.org/558915,United States,William Whitehead,Vireo flavifrons_Yellow-throated Vireo,-74.2495,07:00,William Whitehead,Creative Commons Attribution-NonCommercial-Sha...
23780,4.0,no,yetvir,1 (mono),2020-07-27,41,XC579823.mp3,Yellow-throated Vireo,XC579823 Yellow-throated Vireo (Vireo flavifrons),[],...,NaN,579823,https://www.xeno-canto.org/579823,United States,Bobby Wilcox,Vireo flavifrons_Yellow-throated Vireo,-91.0756,12:00,Bobby Wilcox,Creative Commons Attribution-NonCommercial-Sha...
23781,4.0,no,yetvir,2 (stereo),2020-07-10,25,XC574737.mp3,Yellow-throated Vireo,XC574737 Yellow-throated Vireo (Vireo flavifrons),[],...,NaN,574737,https://www.xeno-canto.org/574737,Canada,Jon Ruddy,Vireo flavifrons_Yellow-throated Vireo,-76.3558,10:30,Jon Ruddy,Creative Commons Attribution-NonCommercial-Sha...
23782,3.5,no,yetvir,1 (mono),2020-07-03,33,XC573213.mp3,Yellow-throated Vireo,XC573213 Yellow-throated Vireo (Vireo flavifrons),['Geothlypis trichas_Common Yellowthroat'],...,Common Yellowthroat (Geothlypis trichas),573213,https://www.xeno-canto.org/573213,United States,Annette McClellan,Vireo flavifrons_Yellow-throated Vireo,-87.8015,06:00,Annette McClellan,Creative Commons Attribution-NonCommercial-Sha...


In [21]:
df = df[df.duration >= 1]
df.reset_index(drop=True, inplace=True)
df

,rating,playback_used,ebird_code,channels,date,duration,filename,species,title,secondary_labels,...,background,xc_id,url,country,author,primary_label,longitude,time,recordist,license
0,0.0,no,aldfly,2 (stereo),2019-06-11,49,XC554809.mp3,Alder Flycatcher,XC554809 Alder Flycatcher (Empidonax alnorum),[],...,NaN,554809,https://www.xeno-canto.org/554809,United States,Ron Overholtz,Empidonax alnorum_Alder Flycatcher,-149.9757,07:00,Ron Overholtz,Creative Commons Attribution-NonCommercial-Sha...
1,0.0,no,aldfly,2 (stereo),2020-04-14,21,XC552408.mp3,Alder Flycatcher,XC552408 Alder Flycatcher (Empidonax alnorum),[],...,NaN,552408,https://www.xeno-canto.org/552408,Colombia,Jerome Fischer,Empidonax alnorum_Alder Flycatcher,-74.6520,17:00,Jerome Fischer,Creative Commons Attribution-NonCommercial-Sha...
2,0.0,yes,aldfly,2 (stereo),2020-04-05,35,XC544552.mp3,Alder Flycatcher,XC544552 Alder Flycatcher (Empidonax alnorum),[],...,NaN,544552,https://www.xeno-canto.org/544552,Colombia,Jerome Fischer,Empidonax alnorum_Alder Flycatcher,-74.6520,07:00,Jerome Fischer,Creative Commons Attribution-NonCommercial-Sha...
3,0.0,yes,aldfly,2 (stereo),2020-04-05,12,XC544551.mp3,Alder Flycatcher,XC544551 Alder Flycatcher (Empidonax alnorum),[],...,NaN,544551,https://www.xeno-canto.org/544551,Colombia,Jerome Fischer,Empidonax alnorum_Alder Flycatcher,-74.6520,07:00,Jerome Fischer,Creative Commons Attribution-NonCommercial-Sha...
4,0.0,yes,aldfly,2 (stereo),2020-04-05,38,XC544550.mp3,Alder Flycatcher,XC544550 Alder Flycatcher (Empidonax alnorum),[],...,NaN,544550,https://www.xeno-canto.org/544550,Colombia,Jerome Fischer,Empidonax alnorum_Alder Flycatcher,-74.6520,07:00,Jerome Fischer,Creative Commons Attribution-NonCommercial-Sha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23739,4.5,no,yetvir,2 (stereo),2020-05-16,63,XC558915.mp3,Yellow-throated Vireo,XC558915 Yellow-throated Vireo (Vireo flavifrons),['Setophaga striata_Blackpoll Warbler'],...,Blackpoll Warbler (Setophaga striata),558915,https://www.xeno-canto.org/558915,United States,William Whitehead,Vireo flavifrons_Yellow-throated Vireo,-74.2495,07:00,William Whitehead,Creative Commons Attribution-NonCommercial-Sha...
23740,4.0,no,yetvir,1 (mono),2020-07-27,41,XC579823.mp3,Yellow-throated Vireo,XC579823 Yellow-throated Vireo (Vireo flavifrons),[],...,NaN,579823,https://www.xeno-canto.org/579823,United States,Bobby Wilcox,Vireo flavifrons_Yellow-throated Vireo,-91.0756,12:00,Bobby Wilcox,Creative Commons Attribution-NonCommercial-Sha...
23741,4.0,no,yetvir,2 (stereo),2020-07-10,25,XC574737.mp3,Yellow-throated Vireo,XC574737 Yellow-throated Vireo (Vireo flavifrons),[],...,NaN,574737,https://www.xeno-canto.org/574737,Canada,Jon Ruddy,Vireo flavifrons_Yellow-throated Vireo,-76.3558,10:30,Jon Ruddy,Creative Commons Attribution-NonCommercial-Sha...
23742,3.5,no,yetvir,1 (mono),2020-07-03,33,XC573213.mp3,Yellow-throated Vireo,XC573213 Yellow-throated Vireo (Vireo flavifrons),['Geothlypis trichas_Common Yellowthroat'],...,Common Yellowthroat (Geothlypis trichas),573213,https://www.xeno-canto.org/573213,United States,Annette McClellan,Vireo flavifrons_Yellow-throated Vireo,-87.8015,06:00,Annette McClellan,Creative Commons Attribution-NonCommercial-Sha...


In [22]:
#array labels

lista_2 = []

dict_num_labels = dict(df.ebird_code.value_counts())
sorted_dict = sorted(dict_num_labels.items())

for i in range(len(sorted_dict)):
    lista_1 = [i for _ in range(sorted_dict[i][1])]
    lista_2 = lista_2 + lista_1

array_label = np.array(lista_2)

In [23]:
len(array_label)

23744

In [24]:
model = torch.hub.load('harritaylor/torchvggish', 'vggish')
model.eval()

Downloading: "https://github.com/harritaylor/torchvggish/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/harritaylor/torchvggish/releases/download/v0.1/vggish-10086976.pth" to /root/.cache/torch/hub/checkpoints/vggish-10086976.pth


  0%|          | 0.00/275M [00:00<?, ?B/s]

Downloading: "https://github.com/harritaylor/torchvggish/releases/download/v0.1/vggish_pca_params-970ea276.pth" to /root/.cache/torch/hub/checkpoints/vggish_pca_params-970ea276.pth


  0%|          | 0.00/177k [00:00<?, ?B/s]

VGGish(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

In [7]:
!pip install pydub

In [8]:
p_1 = '../input/xeno-canto-bird-recordings-extended-a-m/A-M'
p_2 = '../input/xeno-canto-bird-recordings-extended-n-z/N-Z'

In [9]:
from os import path
from pydub import AudioSegment

def embeddingsLabels(df):

    y = []
    emb = []

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        
        
        if row.ebird_code[0] <= 'm':
        
            ruta = f"{p_1}/{row.ebird_code}/{row.filename}"
            
        else:
            
            ruta = f"{p_2}/{row.ebird_code}/{row.filename}"
            
        src = ruta
        dst = "f.wav"

        # convert wav to mp3                                                            
        sound = AudioSegment.from_mp3(src)
        sound.export(dst, format="wav")
        
        embeddings = model('./f.wav')
        
        !rm 'f.wav'
        
        embeddings = torch.mean(embeddings, 0)
        
        emb.append(embeddings.detach().to('cpu').tolist())
        
        del embeddings
        
        #if index == 10:
            #break
        
   
        
    return emb

In [10]:
X = embeddingsLabels(df)

  0%|          | 10/23744 [00:33<22:11:50,  3.37s/it]
